In [69]:
import pandas as pd
import plotly.express as px
import h3

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [70]:
simulation = pd.read_csv('simulation.csv')
print(simulation.shape)
simulation.head()

(7532, 12)


,drifting,steps_in_state,h3_index,month,mean_heading,home_region,fork_length_cm,ptt,date,heading,lat,lon
0,True,1,8422d0bffffffff,12,0.0,Unknown,85.0,129843,2013-12-19,NaN,53.980980,-166.800355
1,True,1,8422d57ffffffff,12,0.0,Unknown,85.0,129843,2013-12-20,0.0,53.852089,-167.386313
2,True,1,8422d57ffffffff,12,0.0,Unknown,85.0,129843,2013-12-21,0.0,53.852089,-167.386313
3,True,1,8422d57ffffffff,12,0.0,Unknown,85.0,129843,2013-12-22,0.0,53.852089,-167.386313
4,True,1,8422d57ffffffff,12,0.0,Unknown,85.0,129843,2013-12-23,0.0,53.852089,-167.386313


In [71]:
state = pd.read_csv('training_states.csv')
state['lat'] = state['h3_index'].apply(lambda x: h3.h3_to_geo(x)[0])
state['lon'] = state['h3_index'].apply(lambda x: h3.h3_to_geo(x)[1])
print(state.shape)
state.head()

(7421, 11)


,ptt,h3_index,month,mean_heading,drifting,steps_in_state,date,fork_length_cm,home_region,lat,lon
0,129843,8422d0bffffffff,12,0.0,True,1.0,2013-12-19,85.0,Unknown,53.980980,-166.800355
1,129843,8422d03ffffffff,12,0.0,True,2.0,2013-12-20,85.0,Unknown,54.360925,-166.742418
2,129843,8422d03ffffffff,12,0.0,True,3.0,2013-12-21,85.0,Unknown,54.360925,-166.742418
3,129843,8422d03ffffffff,12,0.0,True,4.0,2013-12-22,85.0,Unknown,54.360925,-166.742418
4,129843,8422d03ffffffff,12,0.0,True,5.0,2013-12-23,85.0,Unknown,54.360925,-166.742418


In [72]:
def plot_it(data, lat, lon, color):
    fig = px.scatter_geo(
        data, lat=lat, lon=lon, color=color,
    )
    fig.update_layout(autosize=True, height=600, geo=dict(center=dict(lat=58, lon=-150), projection_scale=6))
    return fig

In [73]:
plot_it(state, 'lat', 'lon', 'ptt')

In [74]:
plot_it(simulation, 'lat', 'lon', 'ptt')

In [75]:
groups = pd.read_csv('../train/DriftMovementLeaf_groupings.csv').rename({'_identifier': 'ptt'}, axis=1)
groups['explained_variance_key'] = ((groups['explained_variance'] * 100).astype(int) / 100).astype(str)

In [76]:
grouped_state = state.merge(groups, on='ptt')
plot_it(grouped_state[~grouped_state['drifting']], 'lat', 'lon', 'explained_variance_key')

In [77]:
grouped_simulation = simulation.merge(groups, on='ptt')
plot_it(grouped_simulation[~grouped_simulation['drifting']], 'lat', 'lon', 'explained_variance_key')